In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import os
from mpl_toolkits.mplot3d import axes3d
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
import sklearn.naive_bayes as nb
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from tensorflow.keras.layers import LSTM,Conv1D,MaxPooling1D
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
import tensorflow.keras.backend as K 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout,Input,Dense,Activation,Flatten,SeparableConv2D,BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
import tensorflow as tf

target_folder = 'swing'
mode = 'RandomForest'

print('this model is ',mode,'\n\n')
filename_in_dir = []

for root, dirs, files in os.walk(target_folder):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)
print('load_file_complete')
print('make DataFrame...')
total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
print('make DataFrame complete ....')

print('label_encoding processing...')
encoder = LabelEncoder()
arr_label = encoder.fit_transform(total_label)

total_data = np.array(total_data)
arr_label = np.array(arr_label)
x_train,x_test,y_train,y_test = train_test_split(total_data,arr_label,test_size=0.2,random_state=0)

nsamples, nx, ny = x_train.shape
d2_train_dataset = x_train.reshape((nsamples,nx*ny))

nsamples, nx, ny = x_test.shape
d2_test_dataset = x_test.reshape((nsamples,nx*ny))


#####here to change####
######################################################################
rfc=RandomForestClassifier(random_state=42)
param_grid = { 
    'n_estimators': [10,15,20,30,40,50,100,200,300,500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [3,4,5,6,7,8,9,10],
    'criterion' :['gini', 'entropy']
}

mod = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 10,n_jobs=-1)
######################################################################


predict_model = mod.fit(d2_train_dataset,y_train)
print('model best parameter : ',predict_model.best_params_)
print('fitting ',mode,' is complete...')
print(mode,'score is :',predict_model.score(d2_test_dataset,y_test))

prediction = predict_model.predict(d2_test_dataset)



hit = 0
miss = 0
for x in range(len(y_test)):
    if prediction[x] == y_test[x]:
        hit+=1
    else:
        miss+=1

print('Validation Result...')
print('hit: ',hit,' miss : ',miss)

print('\n\nnow test new data..')

filename_in_dir = []

for root, dirs, files in os.walk('penholder/'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)

total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
    
total_data = np.array(total_data)
nsamples, nx, ny = total_data.shape
sample = total_data.reshape((nsamples,nx*ny))

sample_pred = predict_model.predict(sample)

lab = encoder.inverse_transform(sample_pred)

hit = 0
miss = 0
answer=[]
print('testing new data result :\n[answer]  -->  [predict err]')
for x in range(len(lab)):
    if lab[x] == total_label[x]:
        hit+=1
        answer.append(lab[x])
    else:
        miss+=1
        print(total_label[x],' --> ' ,lab[x],'err_index : ',x)

print('hit: ',hit,' miss : ',miss)

/home/younghwan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/younghwan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/younghwan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/younghwan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dty

this model is  RandomForest 


load_file_complete
make DataFrame...
make DataFrame complete ....
label_encoding processing...
model best parameter :  {'criterion': 'entropy', 'max_depth': 10, 'max_features': 'log2', 'n_estimators': 300}
fitting  RandomForest  is complete...
RandomForest score is : 0.9767981438515081
Validation Result...
hit:  842  miss :  20


now test new data..
testing new data result :
[answer]  -->  [predict err]
fo_drive  -->  back_cut err_index :  0
fo_drive  -->  back_cut err_index :  1
fo_drive  -->  back_cut err_index :  4
fo_drive  -->  back_cut err_index :  6
fo_drive  -->  back_cut err_index :  7
fo_drive  -->  back_cut err_index :  9
fo_drive  -->  back_cut err_index :  10
fo_drive  -->  back_cut err_index :  11
fo_drive  -->  back_cut err_index :  12
fo_drive  -->  back_cut err_index :  16
fo_drive  -->  back_cut err_index :  17
back_drive  -->  back_cut err_index :  18
back_drive  -->  fo_smash err_index :  19
back_drive  -->  fo_drive err_index :  20
ba